<a href="https://colab.research.google.com/github/Sulin7254/Python/blob/main/colab/conditional_generation/tev/ProteinGenerator_conditional.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

!wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
!chmod +x Miniconda3-latest-Linux-x86_64.sh
!./Miniconda3-latest-Linux-x86_64.sh -b -f -p /usr/local

!git clone https://github.com/RosettaCommons/protein_generator

Mounted at /content/drive
--2025-06-24 13:30:10--  https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.32.241, 104.16.191.158, 2606:4700::6810:20f1, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.32.241|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 155472915 (148M) [application/octet-stream]
Saving to: ‘Miniconda3-latest-Linux-x86_64.sh’

Miniconda3-latest-L 100%[===================>] 148.27M   122MB/s    in 1.2s    

2025-06-24 13:30:11 (122 MB/s) - ‘Miniconda3-latest-Linux-x86_64.sh’ saved [155472915/155472915]

PREFIX=/usr/local
Unpacking payload ...
entry_point.py:256: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the filter argument to control this behavior.
entry_point.py:256: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify 

In [2]:
%cd ./protein_generator
!conda env create -f environment.yml

/content/protein_generator
Channels:
 - dglteam/label/cu117
 - nvidia
 - pytorch
 - bioconda
 - defaults
 - conda-forge
 - r
Platform: linux-64
Solving environment: / - done


==> WARNING: A newer version of conda exists. <==
    current version: 25.3.1
    latest version: 25.5.1

Please update conda by running

    $ conda update -n base -c defaults conda



pytorch-2.0.0        | 1.41 GB   | :   0% 0/1 [00:00<?, ?it/s]
libcublas-11.11.3.6  | 364.0 MB  | :   0% 0/1 [00:00<?, ?it/s]

dgl-1.0.2.cu117      | 228.2 MB  | :   0% 0/1 [00:00<?, ?it/s]


libcusparse-11.7.5.8 | 176.3 MB  | :   0% 0/1 [00:00<?, ?it/s]



mkl-2023.1.0         | 171.5 MB  | :   0% 0/1 [00:00<?, ?it/s]




libnpp-11.8.0.86     | 147.8 MB  | :   0% 0/1 [00:00<?, ?it/s]





libcufft-10.9.0.58   | 142.8 MB  | :   0% 0/1 [00:00<?, ?it/s]






libcusolver-11.4.1.4 | 96.5 MB   | :   0% 0/1 [00:00<?, ?it/s]







torchtriton-2.0.0    | 62.6 MB   | :   0% 0/1 [00:00<?, ?it/s]








libcurand-10.3.10.19 | 44.0 MB 

In [3]:
%%bash
source activate proteingenerator
pip install -e ./

Obtaining file:///content/protein_generator
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Running setup.py develop for protein_generator


  DEPRECATION: Legacy editable install of protein_generator==1.1.0 from file:///content/protein_generator (setup.py develop) is deprecated. pip 25.0 will enforce this behaviour change. A possible replacement is to add a pyproject.toml or enable --use-pep517, and use setuptools >= 64. If the resulting installation is not behaving as expected, try using --config-settings editable_mode=compat. Please consult the setuptools documentation for more information. Discussion can be found at https://github.com/pypa/pip/issues/11457


In [4]:
!wget http://files.ipd.uw.edu/pub/sequence_diffusion/checkpoints/SEQDIFF_230205_dssp_hotspots_25mask_EQtasks_mod30.pt
!wget http://files.ipd.uw.edu/pub/sequence_diffusion/checkpoints/SEQDIFF_221219_equalTASKS_nostrSELFCOND_mod30.pt

--2025-06-24 13:36:50--  http://files.ipd.uw.edu/pub/sequence_diffusion/checkpoints/SEQDIFF_230205_dssp_hotspots_25mask_EQtasks_mod30.pt
Resolving files.ipd.uw.edu (files.ipd.uw.edu)... 128.95.160.135, 128.95.160.134, 2607:4000:406::160:135, ...
Connecting to files.ipd.uw.edu (files.ipd.uw.edu)|128.95.160.135|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 965589513 (921M) [application/octet-stream]
Saving to: ‘SEQDIFF_230205_dssp_hotspots_25mask_EQtasks_mod30.pt’

SEQDIFF_230205_dssp 100%[===================>] 920.86M  6.77MB/s    in 95s     

2025-06-24 13:38:25 (9.74 MB/s) - ‘SEQDIFF_230205_dssp_hotspots_25mask_EQtasks_mod30.pt’ saved [965589513/965589513]

--2025-06-24 13:38:25--  http://files.ipd.uw.edu/pub/sequence_diffusion/checkpoints/SEQDIFF_221219_equalTASKS_nostrSELFCOND_mod30.pt
Resolving files.ipd.uw.edu (files.ipd.uw.edu)... 128.95.160.134, 128.95.160.135, 2607:4000:406::160:134, ...
Connecting to files.ipd.uw.edu (files.ipd.uw.edu)|128.95.160.134

In [5]:
import os
import shutil
import glob
import json
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import uuid
from datetime import datetime
import re
import torch
from time import time

meta_data_filepath = "/content/drive/MyDrive/Generative_Models/conditional_generation/protein_generator_tev/generation_metadata_proteingenerator_tev.csv"

if os.path.exists(meta_data_filepath):
  all_metadata_df = pd.read_csv(meta_data_filepath)
  print("Existing generation metadata read in.")
else:
  all_metadata_df = pd.DataFrame()
  #all_metadata_df.to_csv(meta_data_filepath, index=False)
  print("Created generation metadata dataframe")


Created generation metadata dataframe


In [7]:
from google.colab import drive
drive.mount('/content/drive')

# 然后创建目录（只需运行一次）
import os
output_dir = '/content/drive/MyDrive/Generative_Models/conditional_generation/protein_generator_tev'
os.makedirs(output_dir, exist_ok=True)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
import time
!mkdir samples


generation_command = f"""
source activate proteingenerator
python ./inference.py --num_designs 50 --out samples/scaffolding --pdb /content/drive/MyDrive/Generative_Models/conditional_generation/tev_monomer.pdb  --contigs 27,A28-33,13,A47-51,88,A140-152,15,A168-179,32,A212-221,16 --T 25 --save_best_plddt
"""
#generation_command = f"""
#source activate proteingenerator
#python ./inference.py --num_designs 50 --out samples/scaffolding --sequence XXXXXXXXXXXXXXXXXXXXXXXXXXXGHTTSLXXXXXXXXXXXXXHLFRRXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXFWKHWIQTKDGQCXXXXXXXXXXXXXXXHSASNFTNTNNYXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXWGGHKVFMVKXXXXXXXXXXXXXXXX --T 25 --save_best_plddt
#"""


batch_size = 50
meta_data = {}
meta_data['batch_id'] = str(uuid.uuid4())
meta_data['batch_size'] = str(batch_size)
meta_data['Timestamp'] = str(datetime.now())
meta_data['model'] = 'protein_generator'
meta_data['task'] = 'all_atom_pdb_generation'
meta_data['conditions'] = 'tev (monomer) scaffolding (structure) [--contigs 27,A28-33,13,A47-51,88,A140-152,15,A168-179,32,A212-221,16]'
#meta_data['conditions'] = 'tev (monomer) scaffolding (sequence) [--sequence XXXXXXXXXXXXXXXXXXXXXXXXXXXGHTTSLXXXXXXXXXXXXXHLFRRXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXFWKHWIQTKDGQCXXXXXXXXXXXXXXXHSASNFTNTNNYXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXWGGHKVFMVKXXXXXXXXXXXXXXXX]'
meta_data['gpu'] = 'T4 GPU'
start_time = time.time()
!{generation_command}
end_time = time.time()
total_job_time = end_time - start_time
meta_data['wall_time_batch'] = str(total_job_time) + " Seconds"
meta_data['wall_time_task'] = str(total_job_time/batch_size) + " Seconds (inferred)"

for filename in os.listdir("/content/protein_generator/samples"):
  if filename.endswith(".pdb"):
    meta_data['entity_id'] = str(uuid.uuid4())
    #new_name = "proteingenerator_tev_seq_" + meta_data['entity_id'] + ".pdb"
    new_name = "proteingenerator_tev_struct_" + meta_data['entity_id'] + ".pdb"
    meta_data['output_file_name'] = new_name
    metadata_entry = pd.Series(meta_data)
    all_metadata_df = pd.concat([all_metadata_df,pd.DataFrame(metadata_entry).T], ignore_index=True)
    cleanup_command = f"""mv /content/protein_generator/samples/{filename} /content/drive/MyDrive/Generative_Models/IL10_scaffolding/protein_generator_il10/{new_name}"""
    !{cleanup_command}

all_metadata_df.to_csv(meta_data_filepath, index=False)
print("Metadata saved. Cleaning up....")
!rm -r /content/protein_generator/samples

torch.cuda.empty_cache()

mkdir: cannot create directory ‘samples’: File exists

            ██████╗ ██████╗  ██████╗ ████████╗███████╗██╗███╗   ██╗
            ██╔══██╗██╔══██╗██╔═══██╗╚══██╔══╝██╔════╝██║████╗  ██║
            ██████╔╝██████╔╝██║   ██║   ██║   █████╗  ██║██╔██╗ ██║
            ██╔═══╝ ██╔══██╗██║   ██║   ██║   ██╔══╝  ██║██║╚██╗██║
            ██║     ██║  ██║╚██████╔╝   ██║   ███████╗██║██║ ╚████║
            ╚═╝     ╚═╝  ╚═╝ ╚═════╝    ╚═╝   ╚══════╝╚═╝╚═╝  ╚═══╝
     ██████╗ ███████╗███╗   ██╗███████╗██████╗  █████╗ ████████╗ ██████╗ ██████╗ 
    ██╔════╝ ██╔════╝████╗  ██║██╔════╝██╔══██╗██╔══██╗╚══██╔══╝██╔═══██╗██╔══██╗
    ██║  ███╗█████╗  ██╔██╗ ██║█████╗  ██████╔╝███████║   ██║   ██║   ██║██████╔╝
    ██║   ██║██╔══╝  ██║╚██╗██║██╔══╝  ██╔══██╗██╔══██║   ██║   ██║   ██║██╔══██╗
    ╚██████╔╝███████╗██║ ╚████║███████╗██║  ██║██║  ██║   ██║   ╚██████╔╝██║  ██║
     ╚═════╝ ╚══════╝╚═╝  ╚═══╝╚══════╝╚═╝  ╚═╝╚═╝  ╚═╝   ╚═╝    ╚═════╝ ╚═╝  ╚═╝
        

    
using sampler: default
Traceba